In [12]:
#importing all the packages need 
import seaborn as sns
import pandas as pd
import pandas.api.types as ptypes
import numpy as np
import missingno as mno
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.max_open_warning'] = False 

from sklearn.feature_selection import chi2 

from scipy.stats import chi2_contingency
import re

In [13]:
#reading the data set file
df = pd.read_csv("training_v2.csv")

In [14]:
df.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [15]:
#Dropping all the variables with 60% more misisng values
nacounts = df.isna().sum()
less_than_60 = list(nacounts[nacounts.sort_values() < df.shape[0]*0.6].index)
greater_than_60 = list(nacounts[nacounts.sort_values() > df.shape[0]*0.6].index)
print(len(less_than_60))
print(len(greater_than_60))

120
66


In [16]:
print(greater_than_60)

['bilirubin_apache', 'fio2_apache', 'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache', 'd1_diasbp_invasive_max', 'd1_diasbp_invasive_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min', 'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min', 'h1_diasbp_invasive_max', 'h1_diasbp_invasive_min', 'h1_mbp_invasive_max', 'h1_mbp_invasive_min', 'h1_sysbp_invasive_max', 'h1_sysbp_invasive_min', 'd1_inr_max', 'd1_inr_min', 'd1_lactate_max', 'd1_lactate_min', 'h1_albumin_max', 'h1_albumin_min', 'h1_bilirubin_max', 'h1_bilirubin_min', 'h1_bun_max', 'h1_bun_min', 'h1_calcium_max', 'h1_calcium_min', 'h1_creatinine_max', 'h1_creatinine_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max', 'h1_hemaglobin_min', 'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min', 'h1_lactate_max', 'h1_lactate_min', 'h1_platelets_max', 'h1_platelets_min', 'h1_potassium_max', 'h1_potassium_min', 'h1_sodium_max', 'h1_sodium_min', 'h1_wbc_max', 'h1_wbc_min', 'd1_arterial_pco2_max', 'd1_arteria

In [17]:
new_df = df[nacounts[nacounts.sort_values() < df.shape[0]*0.6].index]

In [18]:
new_df.head()
new_df.shape

(91713, 120)

In [19]:
#Which other columns has missing values
missing_col_list = list(new_df.columns[new_df.isnull().any()])
print(missing_col_list)
print(len(missing_col_list))

new_df.dtypes

['age', 'bmi', 'ethnicity', 'gender', 'height', 'hospital_admit_source', 'icu_admit_source', 'weight', 'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache', 'bun_apache', 'creatinine_apache', 'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 'intubated_apache', 'map_apache', 'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache', 'ventilated_apache', 'wbc_apache', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_max', 'h1_diasbp_min', 'h1_diasbp_noninvasive_max', 'h1_diasbp_noninvasive_min', 'h1_heartrat

encounter_id                     int64
patient_id                       int64
hospital_id                      int64
hospital_death                   int64
age                            float64
                                ...   
leukemia                       float64
lymphoma                       float64
solid_tumor_with_metastasis    float64
apache_3j_bodysystem            object
apache_2_bodysystem             object
Length: 120, dtype: object

Imputation with average, mode , backward filling or forwrad filling

In [20]:
mean_age = round(new_df['age'].mean())
new_df['age'].fillna(mean_age, inplace=True)    

C:\Users\shahr\Anaconda3\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [21]:
new_df.astype({'age': 'int64'}).dtypes

encounter_id                     int64
patient_id                       int64
hospital_id                      int64
hospital_death                   int64
age                              int64
                                ...   
leukemia                       float64
lymphoma                       float64
solid_tumor_with_metastasis    float64
apache_3j_bodysystem            object
apache_2_bodysystem             object
Length: 120, dtype: object

In [22]:
new_df.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [23]:
new_bmi = new_df['weight']/(new_df['height']**2)
new_df['bmi'].fillna(new_bmi, inplace=True) 
median_bmi = (new_df['bmi'].median())
new_df['bmi'].fillna(median_bmi, inplace=True) 

In [24]:
new_df['ethnicity'].fillna(method='ffill', inplace=True)

In [25]:
new_df['gender'].fillna(method='bfill', inplace=True)

In [26]:
mean_height = round(new_df['height'].mean())
new_df['height'].fillna(mean_height, inplace=True) 

In [27]:
#new_weight = new_df['bmi']*(new_df['height']**2)
new_df['weight'].fillna(new_df['weight'].mean(), inplace=True) 

In [28]:
new_df = new_df[new_df['aids'].notna()]
new_df = new_df[new_df['apache_2_diagnosis'].notna()]
new_df.shape

(90051, 120)

In [29]:
new_df['elective_surgery'] = new_df['elective_surgery'].astype("bool")
new_df['apache_post_operative'] = new_df['apache_post_operative'].astype("bool")
new_df['arf_apache'] = new_df['arf_apache'].astype("bool")
new_df['gcs_unable_apache'] = new_df['gcs_unable_apache'].astype("bool")
new_df['intubated_apache'] = new_df['intubated_apache'].astype("bool")
new_df['ventilated_apache'] = new_df['ventilated_apache'].astype("bool")
new_df['aids'] = new_df['aids'].astype("bool")
new_df['cirrhosis'] = new_df['cirrhosis'].astype("bool")
new_df['diabetes_mellitus'] = new_df['diabetes_mellitus'].astype("bool")
new_df['hepatic_failure'] = new_df['hepatic_failure'].astype("bool")
new_df['immunosuppression'] = new_df['immunosuppression'].astype("bool")
new_df['leukemia'] = new_df['leukemia'].astype("bool")
new_df['lymphoma'] = new_df['lymphoma'].astype("bool")
new_df['solid_tumor_with_metastasis'] = new_df['solid_tumor_with_metastasis'].astype("bool")
new_df['hospital_death'] = new_df['hospital_death'].astype("bool")
new_df = pd.get_dummies(new_df, columns=['ethnicity','gender','hospital_admit_source','icu_admit_source','icu_stay_type','icu_type',
                                         'apache_3j_bodysystem','apache_2_bodysystem'])


In [30]:
new_df.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,...,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,66154,25312,118,False,68.0,22.730000,False,180.3,92,0.541667,...,1,0,0,0,0,0,0,0,0,0
1,114252,59342,81,False,77.0,27.420000,False,160.0,90,0.927778,...,0,0,0,0,0,0,1,0,0,0
2,119783,50777,118,False,25.0,31.950000,False,172.7,93,0.000694,...,0,0,0,1,0,0,0,0,0,0
3,79267,46918,118,False,81.0,22.640000,True,165.1,92,0.000694,...,1,0,0,0,0,0,0,0,0,0
4,92056,34377,33,False,19.0,27.654655,False,188.0,91,0.073611,...,0,0,0,0,0,0,0,1,0,0


In [34]:
missing_col_list = list(new_df.columns[new_df.isnull().any()])
print(missing_col_list)
print(len(missing_col_list))

[]
0


In [24]:
missing_var=list(new_df.columns[new_df.isnull().sum()>0])
independent_var = list(new_df.columns[new_df.isnull().sum()==0])#list(new_df.columns)

for ls in ["encounter_id","hospital_id","patient_id"]:
    independent_var.remove(ls)

for ls in missing_var:
#if new_df[ls].isnull().sum() > 0:
    df_with_var = new_df[pd.isnull(new_df[ls]) == False]
    df_without_var = new_df[pd.isnull(new_df[ls])]

    #independent_var.remove(ls)

    rfimpute = RandomForestRegressor()
    rfimpute.fit(df_with_var[independent_var], df_with_var[ls])

    generatedvarValues = rfimpute.predict(X=df_without_var[independent_var])

    df_without_var[ls] = generatedvarValues.astype(new_df[ls].dtype)
    data = df_with_var.append(df_without_var)

    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)

    new_df=data.copy()
    

C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


KeyboardInterrupt: 

In [32]:
new_df.isna().any().sum()

85

In [24]:
one_more_copy_new_df = new_df

In [33]:
numeric_cols = [col for col in new_df.columns if ptypes.is_numeric_dtype(new_df[col])]
print(numeric_cols)

for i in new_df[numeric_cols]:
    new_df[i].fillna(new_df[i].mean(),inplace = True)

new_df[numeric_cols].isna().any()

#new_df['hospital_admit_source'].fillna(new_df['hospital_admit_source'].max(),inplace = True)

['encounter_id', 'patient_id', 'hospital_id', 'hospital_death', 'age', 'bmi', 'elective_surgery', 'height', 'icu_id', 'pre_icu_los_days', 'readmission_status', 'weight', 'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'apache_post_operative', 'arf_apache', 'bun_apache', 'creatinine_apache', 'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 'intubated_apache', 'map_apache', 'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache', 'ventilated_apache', 'wbc_apache', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_diasbp_noninvasive_max', 'd1_diasbp_noninvasive_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_m

encounter_id                               False
patient_id                                 False
hospital_id                                False
hospital_death                             False
age                                        False
                                           ...  
apache_2_bodysystem_Renal/Genitourinary    False
apache_2_bodysystem_Respiratory            False
apache_2_bodysystem_Trauma                 False
apache_2_bodysystem_Undefined Diagnoses    False
apache_2_bodysystem_Undefined diagnoses    False
Length: 172, dtype: bool

In [35]:
from sklearn.model_selection import train_test_split
colsToInclude = new_df.columns
colsToExclude = ["hospital_death"]
colsToExclude += [ c for c in new_df.columns if c not in colsToInclude]
#df["intercept"] = 1
Y = new_df["hospital_death"]
X = new_df.drop(columns = colsToExclude)


In [36]:
X.head()

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,...,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,66154,25312,118,68.0,22.730000,False,180.3,92,0.541667,0,...,1,0,0,0,0,0,0,0,0,0
1,114252,59342,81,77.0,27.420000,False,160.0,90,0.927778,0,...,0,0,0,0,0,0,1,0,0,0
2,119783,50777,118,25.0,31.950000,False,172.7,93,0.000694,0,...,0,0,0,1,0,0,0,0,0,0
3,79267,46918,118,81.0,22.640000,True,165.1,92,0.000694,0,...,1,0,0,0,0,0,0,0,0,0
4,92056,34377,33,19.0,27.654655,False,188.0,91,0.073611,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 90 attributes
rfe = RFE(model, 90)
rfe = rfe.fit(X, Y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
cols = list(X.columns)
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
df_temp = new_df[selected_features_rfe]
df_temp.head()

In [ ]:
Y = new_df["hospital_death"]
X = df_temp

new_df = new_df[['age',
 'elective_surgery',
 'pre_icu_los_days',
 'albumin_apache',
 'arf_apache',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_unable_apache',
 'ventilated_apache',
 'd1_spo2_min',
 'd1_temp_max',
 'd1_temp_min',
 'h1_resprate_min',
 'h1_temp_max',
 'h1_temp_min',
 'd1_albumin_min',
 'd1_bilirubin_max',
 'd1_calcium_max',
 'd1_calcium_min',
 'd1_creatinine_max',
 'd1_creatinine_min',
 'd1_hco3_max',
 'd1_hco3_min',
 'd1_hemaglobin_max',
 'd1_hemaglobin_min',
 'd1_hematocrit_max',
 'd1_hematocrit_min',
 'd1_potassium_max',
 'apache_4a_hospital_death_prob',
 'apache_4a_icu_death_prob',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis',
 'ethnicity_Caucasian',
 'ethnicity_Hispanic',
 'ethnicity_Native American',
 'ethnicity_Other/Unknown',
 'gender_F',
 'gender_M',
 'hospital_admit_source_Acute Care/Floor',
 'hospital_admit_source_Chest Pain Center',
 'hospital_admit_source_Direct Admit',
 'hospital_admit_source_Emergency Department',
 'hospital_admit_source_ICU',
 'hospital_admit_source_Observation',
 'hospital_admit_source_Operating Room',
 'hospital_admit_source_Other',
 'hospital_admit_source_Other Hospital',
 'hospital_admit_source_Other ICU',
 'hospital_admit_source_PACU',
 'hospital_admit_source_Step-Down Unit (SDU)',
 'icu_admit_source_Accident & Emergency',
 'icu_admit_source_Floor',
 'icu_admit_source_Other Hospital',
 'icu_admit_source_Other ICU',
 'icu_stay_type_admit',
 'icu_stay_type_readmit',
 'icu_stay_type_transfer',
 'icu_type_CCU-CTICU',
 'icu_type_CTICU',
 'icu_type_Cardiac ICU',
 'icu_type_MICU',
 'icu_type_Med-Surg ICU',
 'icu_type_Neuro ICU',
 'icu_type_SICU',
 'apache_3j_bodysystem_Cardiovascular',
 'apache_3j_bodysystem_Gastrointestinal',
 'apache_3j_bodysystem_Genitourinary',
 'apache_3j_bodysystem_Gynecological',
 'apache_3j_bodysystem_Hematological',
 'apache_3j_bodysystem_Metabolic',
 'apache_3j_bodysystem_Musculoskeletal/Skin',
 'apache_3j_bodysystem_Neurological',
 'apache_3j_bodysystem_Respiratory',
 'apache_3j_bodysystem_Sepsis',
 'apache_3j_bodysystem_Trauma',
 'apache_2_bodysystem_Cardiovascular',
 'apache_2_bodysystem_Gastrointestinal',
 'apache_2_bodysystem_Haematologic',
 'apache_2_bodysystem_Metabolic',
 'apache_2_bodysystem_Neurologic',
 'apache_2_bodysystem_Respiratory',
 'apache_2_bodysystem_Trauma',
 'apache_2_bodysystem_Undefined Diagnoses',
 'apache_2_bodysystem_Undefined diagnoses',
                'hospital_death']]

Random Forest Model

In [77]:
#Rnaomd Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix ,roc_curve,roc_auc_score ,precision_score, recall_score
from sklearn import metrics

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y ,test_size=0.33, stratify=Y)
y_train.value_counts(normalize=True)

False    0.913614
True     0.086386
Name: hospital_death, dtype: float64

In [32]:
rf = RandomForestClassifier()
rf.fit(X_train , y_train)
roc_auc_score(y_test , rf.predict_proba(X_test)[:,1])


importances = pd.DataFrame({"imp"  : rf.feature_importances_ , "feature" : X_train.columns})
importances.sort_values("imp" , ascending=False).head(60)

list(importances.sort_values("imp" , ascending=False).head(60)["feature"])

importances[importances["feature"] == "cluster"]

C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,imp,feature


In [33]:
y_pred = rf.predict(X_test)

In [34]:
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy for Random Forest Classifier: %.2f%%" % (accuracy * 100.0))

Accuracy for Random Forest Classifier: 92.29%


In [35]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)

In [36]:
print('Random Forest Classifier: ROC AUC=%.3f' % (auc))

Random Forest Classifier: ROC AUC=0.604


Logistic Regression

In [37]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

In [38]:
logit = LogisticRegression(max_iter = 10000)
logit.fit(X_train , y_train)

C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
preds = logit.predict(X_test)
probs_y=logit.predict_proba(X_test)[:,1]

In [40]:
fpr, tpr, thresholds = metrics.roc_curve(y_test , probs_y ,drop_intermediate=False, pos_label=1)
rocdf = pd.DataFrame({"fpr" : fpr , "tpr" : tpr , "thresholds" : thresholds})

In [41]:
from plotnine import *
(ggplot(rocdf , aes(x='fpr' , y='tpr', fill='thresholds')) + geom_line())

accuracy = accuracy_score(y_test , preds)
print("Accuracy for Logistic Regression :", accuracy)

Accuracy for Logistic Regression : 0.9245213177642427


In [42]:
auc = roc_auc_score(y_test , probs_y)
print('Logistic Regression: ROC AUC=%.3f' % (auc))

Logistic Regression: ROC AUC=0.872


# Recursive Feature Elimination

In [44]:
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 90 attributes
rfe = RFE(model, 90)
rfe = rfe.fit(X, Y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\shahr\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[False False False  True False  True False False  True False False  True
 False False False  True False False  True  True  True False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False  True False
 False False False  True  True False False False False False False False
 False False False False  True False False False False False False  True
  True False  True  True False False False  True  True  True  True False
 False  True  True  True  True  True  True False False  True False False
 False False False False False  True  True  True  True  True  True  True
  True  True  True False  True  True  True False  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
 False  True False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [45]:
cols = list(X.columns)
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

Index(['age', 'elective_surgery', 'pre_icu_los_days', 'albumin_apache',
       'arf_apache', 'gcs_eyes_apache', 'gcs_motor_apache',
       'gcs_unable_apache', 'ventilated_apache', 'd1_spo2_min', 'd1_temp_max',
       'd1_temp_min', 'h1_resprate_min', 'h1_temp_max', 'h1_temp_min',
       'd1_albumin_min', 'd1_bilirubin_max', 'd1_calcium_max',
       'd1_calcium_min', 'd1_creatinine_max', 'd1_creatinine_min',
       'd1_hco3_max', 'd1_hco3_min', 'd1_hemaglobin_max', 'd1_hemaglobin_min',
       'd1_hematocrit_max', 'd1_hematocrit_min', 'd1_potassium_max',
       'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', 'aids',
       'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis', 'ethnicity_Asian', 'ethnicity_Caucasian',
       'ethnicity_Hispanic', 'ethnicity_Other/Unknown', 'gender_F', 'gender_M',
       'hospital_admit_source_Acute Care/Floor',
       'hospital_admit_source_Chest Pain Ce

In [48]:
df_temp = new_df[selected_features_rfe]
df_temp.head()

,age,elective_surgery,pre_icu_los_days,albumin_apache,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,ventilated_apache,d1_spo2_min,...,apache_3j_bodysystem_Trauma,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,68.0,False,0.541667,2.300000,False,3.000000,6.000000,False,False,74.0,...,0,1,0,0,0,0,0,0,0,0
1,77.0,False,0.927778,2.902779,False,1.000000,3.000000,False,True,70.0,...,0,0,0,0,0,0,1,0,0,0
2,25.0,False,0.000694,2.902779,False,3.000000,6.000000,False,False,91.0,...,0,0,0,0,1,0,0,0,0,0
3,81.0,True,0.000694,2.902779,False,4.000000,6.000000,False,True,95.0,...,0,1,0,0,0,0,0,0,0,0
4,19.0,False,0.073611,2.902779,False,3.463874,5.469321,True,False,96.0,...,1,0,0,0,0,0,0,1,0,0


In [53]:
Y = new_df["hospital_death"]
X = df_temp

In [54]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X)
# Apply transform to both the training set and the test set.
X = scaler.transform(X)

In [62]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(.95)

In [63]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [64]:
pca.n_components_ 

60

In [69]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [70]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [71]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [73]:
y_pred = model.predict(X_test)

In [74]:
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.29%


In [75]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)

In [76]:
print('XGBoost: ROC AUC=%.3f' % (auc))

XGBoost: ROC AUC=0.637
